# Things to keep in mind

- All communication between stateful functions happens via protobuf structures. The data from Kafka is therefor also in protobuf format. Define your custom structures in the `protobuf/messages.proto` file. Every change to this file requires a recompilation with `protoc`. We included a vscode task (`generate protobuf`) which does this for you.

    **Windows users** will need to change the line endings in scripts/generate_protobuf.sh manually by opening the file and clicking on the CRLF button on the bottom right. You will receive a pop-up, choose the LF option and save the file.

- Messages in Kafka can have a key / value property. The messages written to a topic must have a key (encoded string). Stateful functions keep state according to the key value. This means that messages with the same key will use the same state.
- You need to modify `module.yaml` to register stateful functions, Kafka ingress and egress topics. Modifying the module file requires a complete rebuild of the flink-master and flink-worker Docker images. Flink needs the information from this file to find out how to contact the functions, ingresses and egresses.
- When testing functions, you can use `print()` statements.

The Python SDK can be found [here](https://ci.apache.org/projects/flink/flink-statefun-docs-release-2.2/sdk/python.html).

# How to run/test your code

- If you modified the `module.yaml` file, delete the previous master and worker image (they will have the same docker image id) with `docker rmi --force $IMAGEID`.
- If you modified the `messages.proto` file, recompile the protobuf files and copy the generated code to the directories that use them. For convenience we have added a VSCode task (`Generate protobuf`) which will automate this for this exercise. Use `protoc -I=$PROTOBUF_DIR --python_out=$PROTOBUF_DIR $PROTO_FILE_PATH` to compile manually.
- Start the Flask server.
- Start the Flink master/worker and Kafka cluster with Docker compose. Open a shell (Ctrl + ~) and type `docker-compose up`. **MacOS** users needs to use a terminal on their host.
- Write events with your generator.
- Don't forget to cleanup any remaining docker containers after stopping Docker compose. You can view all containers (stopped and running) with `docker ps -a`. Use `docker rm $CONTAINER_NAME` to remove them.



In [ ]:
from statefun import (
    StatefulFunctions,
    RequestReplyHandler,
    kafka_egress_record,
)

functions = StatefulFunctions()
handler = RequestReplyHandler(functions)

########################
#    YOUR CODE HERE    #
########################

from flask import (
    request,
    make_response,
    Flask,
)

app = Flask(__name__)


@app.route('/statefun', methods=['POST'])
def handle():
    response_data = handler(request.data)
    response = make_response(response_data)
    response.headers.set('Content-Type', 'application/octet-stream')
    return response

if __name__ == '__main__':
    app.run(host="0.0.0.0")